In [1]:
from synthpop.census_helpers import Census
from synthpop import categorizer as cat
from synthpop import synthesizer
import pandas as pd
import numpy as np
import json
import random

In [2]:
def age_cat(r):
    if r.AGEP <= 19: return "19 and under"
    elif r.AGEP <= 35: return "20 to 35"
    elif r.AGEP <= 60: return "35 to 60"
    return "above 60"

def race_cat(r):
    if r.RAC1P == 1: return "white"
    elif r.RAC1P == 2: return "black"
    elif r.RAC1P == 6: return "asian"
    return "other"

def sex_cat(r):
    if r.SEX == 1: return "male"
    return "female"

def tenure_cat(r):
    if r.TEN ==1 or r.TEN ==2: return "owned"
    elif r.TEN ==3:  return "rented"
    return "other"

#Household
def cars_cat(r):
    if r.VEH == 0: return "none"
    elif r.VEH == 1: return "one"
    return "two or more"

def children_cat(r):
    if r.NOC > 0: return "yes"
    return "no"

def income_cat(r): # the example file used FINCP (family income) but the ACS control is Household income
    if r.HINCP > 100000: return "gt100"
    elif r.HINCP > 35000: return "gt35-lt100"
    return "lt35"

def workers_cat(r):
    if r.WIF == 3: return "two or more"
    elif r.WIF == 2: return "two or more"
    elif r.WIF == 1: return "one"
    return "none"

## 设定所需要的普查字段

In [5]:
sample_factor=10 # for the simulation population

city='Detroit'

if city=='Boston': 
    state='25' 
    state_code='ma'
elif city=='Detroit': 
    state='26'
    state_code='mi'
    
ALL_ZONES_PATH='./scripts/cities/'+city+'/clean/model_area.geojson'
SIM_ZONES_PATH='./scripts/cities/'+city+'/clean/sim_zones.json'
OD_PATH='./scripts/cities/'+city+'/raw/LODES/'+state_code+'_od_main_JT00_2015.csv'
ALL_SYNTH_HH_PATH='./scripts/cities/'+city+'/clean/all_synth_hh.csv'
ALL_SYNTH_PERSONS_PATH='./scripts/cities/'+city+'/clean/all_synth_persons.csv'
SIM_POP_PATH='./scripts/cities/'+city+'/clean/sim_pop.json'
VACANT_PATH='./scripts/cities/'+city+'/clean/vacant.json'
FLOATING_PATH='./scripts/cities/'+city+'/clean/floating.json'

c = Census('7a25a7624075d46f112113d33106b6648f42686a')

# load the block group geojson for the whole area
# get set of tracts covered

# identify the data we want at tract and block group level

#Households
income_columns = ['B19001_0%02dE'%i for i in range(1, 18)]
vehicle_columns = ['B08201_0%02dE'%i for i in range(1, 7)]
workers_columns = ['B08202_0%02dE'%i for i in range(1, 6)]
families_columns = ['B11001_001E', 'B11001_002E']
# year_built_columns= ['B25034_001E', 'B25034_002E', 'B25034_003E'] 
# includes vacant structures?
tenure_columns=['B25063_001E', 'B25075_001E']
block_group_columns = income_columns + families_columns + tenure_columns
tract_columns = vehicle_columns + workers_columns

#Persons
population = ['B01001_001E']
sex = ['B01001_002E', 'B01001_026E']
race = ['B02001_0%02dE'%i for i in range(1,11)]
male_age_columns = ['B01001_0%02dE'%i for i in range(3,26)]
female_age_columns = ['B01001_0%02dE'%i for i in range(27,50)]
# worker_class_columns=['B08128_0%02dE'%i for i in range(1, 7)]
worker_class_columns=['B24080_0%02dE'%i for i in [1,3,6,7,8,9,10,13,16,17,18,19,20]]
# One more column needed for the 16 and under not incuded in the worker_class population
all_columns = population + sex + male_age_columns + female_age_columns  + race
# +worker_class_columns

print('家庭层面需要的人口普查字段：\n----------------------------------------------\n{}'.format(block_group_columns))
print('\n\n个体层面需要的人口普查字段：\n----------------------------------------------\n{}'.format(all_columns))

家庭层面需要的人口普查字段：
----------------------------------------------
['B19001_001E', 'B19001_002E', 'B19001_003E', 'B19001_004E', 'B19001_005E', 'B19001_006E', 'B19001_007E', 'B19001_008E', 'B19001_009E', 'B19001_010E', 'B19001_011E', 'B19001_012E', 'B19001_013E', 'B19001_014E', 'B19001_015E', 'B19001_016E', 'B19001_017E', 'B11001_001E', 'B11001_002E', 'B25063_001E', 'B25075_001E']


个体层面需要的人口普查字段：
----------------------------------------------
['B01001_001E', 'B01001_002E', 'B01001_026E', 'B01001_003E', 'B01001_004E', 'B01001_005E', 'B01001_006E', 'B01001_007E', 'B01001_008E', 'B01001_009E', 'B01001_010E', 'B01001_011E', 'B01001_012E', 'B01001_013E', 'B01001_014E', 'B01001_015E', 'B01001_016E', 'B01001_017E', 'B01001_018E', 'B01001_019E', 'B01001_020E', 'B01001_021E', 'B01001_022E', 'B01001_023E', 'B01001_024E', 'B01001_025E', 'B01001_027E', 'B01001_028E', 'B01001_029E', 'B01001_030E', 'B01001_031E', 'B01001_032E', 'B01001_033E', 'B01001_034E', 'B01001_035E', 'B01001_036E', 'B01001_037E', 'B

## 查询获取普查数据 

In [9]:
#overall zones file
all_zones=json.load(open(ALL_ZONES_PATH))
#sim zones file
sim_zones=json.load(open(SIM_ZONES_PATH))

# =============================================================================
# # Get the ACS data and create the person and HH dataframes
# =============================================================================

# loop through the tracts, getting the data for each and appending to the dataframes
all_counties=list(set([f['properties']['COUNTY'] for f in all_zones['features']]))
h_acs=pd.DataFrame()
p_acs=pd.DataFrame()

for county in all_counties:
    h_acs_c = c.block_group_and_tract_query(block_group_columns,
                    tract_columns, state, county, 
                    merge_columns=['tract', 'county', 'state'],
                    block_group_size_attr="B11001_001E",
                    tract_size_attr="B08201_001E",
                    tract=None, year=2017)
#     print(h_acs_c.head())
#     print(h_acs_c.shape)
    h_acs=pd.concat([h_acs, h_acs_c])
    p_acs_c = c.block_group_query(all_columns, state, county, tract=None, year=2017)
#     print(p_acs_c.head())
#     print(p_acs_c.shape)
    
    p_acs=pd.concat([p_acs, p_acs_c])
h_acs=h_acs.reset_index(drop=True)
p_acs=p_acs.reset_index(drop=True)

# add puma information to the dataframes
h_acs['puma'] = h_acs.apply(lambda row: c.tract_to_puma(row['state'], row['county'], row['tract'])[0], axis=1)
p_acs['puma'] = p_acs.apply(lambda row: c.tract_to_puma(row['state'], row['county'], row['tract'])[0], axis=1)


# Since we queried by county, some of the block_groups may not be in our study area
# remove any entries not in those block groups
all_block_group_ids=list(set([f['properties']['TRACT']+'_'+ 
                              f['properties']['BLKGRP']
                              for f in all_zones['features']]))
h_acs['block_group_id']=h_acs.apply(lambda row: 
    row['tract']+'_'+row['block group'], axis=1)
p_acs['block_group_id']=p_acs.apply(lambda row: 
    row['tract']+'_'+row['block group'], axis=1)
h_acs=h_acs.loc[h_acs['block_group_id'].isin(all_block_group_ids)] 
p_acs=p_acs.loc[p_acs['block_group_id'].isin(all_block_group_ids)] 

In [10]:
print('家庭层面按普查小区的统计表：')
h_acs.head()

家庭层面按普查小区的统计表：


,B11001_001E,B11001_002E,B19001_001E,B19001_002E,B19001_003E,B19001_004E,B19001_005E,B19001_006E,B19001_007E,B19001_008E,...,B08201_004E,B08201_005E,B08201_006E,B08202_001E,B08202_002E,B08202_003E,B08202_004E,B08202_005E,puma,block_group_id
0,684,371,684,33,30,71,62,41,11,47,...,229,51,26,684,182,387,80,34,03205,583900_3
1,358,213,358,70,13,51,71,32,8,39,...,120,26,14,358,95,202,41,18,03205,583900_2
2,606,401,606,21,20,22,45,18,19,41,...,203,45,23,606,161,343,70,30,03205,583900_1
3,223,167,223,0,0,4,18,20,14,37,...,86,30,1,223,32,118,57,14,03210,500200_3
4,229,146,229,34,0,10,0,0,31,0,...,88,30,1,229,32,121,59,15,03210,500200_2


In [11]:
print('个体层面按普查小区的统计表：')
p_acs.head()

个体层面按普查小区的统计表：


,B01001_001E,B01001_002E,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,...,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B02001_007E,B02001_008E,B02001_009E,B02001_010E,puma,block_group_id
0,1396,766,33,60,30,54,39,13,0,45,...,675,0,31,0,0,10,0,10,03205,583900_3
1,847,350,29,10,64,0,0,0,0,0,...,103,0,79,0,0,18,0,18,03205,583900_2
2,1572,630,33,17,89,14,9,0,29,0,...,160,0,0,0,0,13,0,13,03205,583900_1
3,668,297,0,13,34,32,5,11,12,18,...,601,0,44,0,0,0,0,0,03210,500200_3
4,592,341,0,78,43,0,0,0,0,26,...,547,0,0,0,0,24,0,24,03210,500200_2


## 类别化

In [12]:
h_acs_cat = cat.categorize(h_acs, {
#     ("households", "total"): "B11001_001E",
    ("children", "yes"): "B11001_002E",
    ("children", "no"): "B11001_001E - B11001_002E",
    ("income", "lt35"): "B19001_002E + B19001_003E + B19001_004E + "
                        "B19001_005E + B19001_006E + B19001_007E",
    ("income", "gt35-lt100"): "B19001_008E + B19001_009E + "
                        "B19001_010E + B19001_011E + B19001_012E"
                        "+ B19001_013E",
    ("income", "gt100"): "B19001_014E + B19001_015E + B19001_016E"
                        "+ B19001_017E",
    ("cars", "none"): "B08201_002E",
    ("cars", "one"): "B08201_003E",
    ("cars", "two or more"): "B08201_004E + B08201_005E + B08201_006E",
    ("workers", "none"): "B08202_002E",
    ("workers", "one"): "B08202_003E",
    ("workers", "two or more"): "B08202_004E + B08202_005E",
    ("tenure", "owned"): "B25063_001E",
    ("tenure", "rented"): "B25075_001E"
}, index_cols=['NAME'])
h_acs=h_acs.set_index('NAME')
h_acs_cat.head()

cat_name                                           cars                   \
cat_value                                          none  one two or more   
NAME                                                                       
Block Group 3, Census Tract 5839, Wayne County,...   82  293         306   
Block Group 2, Census Tract 5839, Wayne County,...   43  153         160   
Block Group 1, Census Tract 5839, Wayne County,...   73  259         271   
Block Group 3, Census Tract 5002, Wayne County,...   11   93         117   
Block Group 2, Census Tract 5002, Wayne County,...   11   95         119   

cat_name                                           children      income  \
cat_value                                                no  yes  gt100   
NAME                                                                      
Block Group 3, Census Tract 5839, Wayne County,...      313  371     64   
Block Group 2, Census Tract 5839, Wayne County,...      145  213     10   
Block Group 1, Census Tract 5839, Wayne County,...      205  401     76   
Block Group 3, Census Tract 5002, Wayne County,...       56  167      0   
Block Group 2, Census Tract 5002, Wayne County,...       83  146     41   

cat_name                                                           tenure  \
cat_value                                          gt35-lt100 lt35  owned   
NAME                                                                        
Block Group 3, Census Tract 5839, Wayne County,...        372  248    414   
Block Group 2, Census Tract 5839, Wayne County,...        103  245    295   
Block Group 1, Census Tract 5839, Wayne County,...        385  145    179   
Block Group 3, Census Tract 5002, Wayne County,...        167   56    136   
Block Group 2, Census Tract 5002, Wayne County,...        113   75    184   

cat_name                                                  workers       \
cat_value                                          rented    none  one   
NAME                                                                     
Block Group 3, Census Tract 5839, Wayne County,...    270     182  387   
Block Group 2, Census Tract 5839, Wayne County,...     63      95  202   
Block Group 1, Census Tract 5839, Wayne County,...    427     161  343   
Block Group 3, Census Tract 5002, Wayne County,...     87      32  118   
Block Group 2, Census Tract 5002, Wayne County,...     45      32  121   

cat_name                                                        
cat_value                                          two or more  
NAME                                                            
Block Group 3, Census Tract 5839, Wayne County,...         114  
Block Group 2, Census Tract 5839, Wayne County,...          59  
Block Group 1, Census Tract 5839, Wayne County,...         100  
Block Group 3, Census Tract 5002, Wayne County,...          71  
Block Group 2, Census Tract 5002, Wayne County,...          74

In [13]:
p_acs_cat = cat.categorize(p_acs, {
#     ("population", "total"): "B01001_001E",
    ("age", "19 and under"): "B01001_003E + B01001_004E + B01001_005E + "
                             "B01001_006E + B01001_007E + B01001_027E + "
                             "B01001_028E + B01001_029E + B01001_030E + "
                             "B01001_031E",
    ("age", "20 to 35"): "B01001_008E + B01001_009E + B01001_010E + "
                         "B01001_011E + B01001_012E + B01001_032E + "
                         "B01001_033E + B01001_034E + B01001_035E + "
                         "B01001_036E",
    ("age", "35 to 60"): "B01001_013E + B01001_014E + B01001_015E + "
                         "B01001_016E + B01001_017E + B01001_037E + "
                         "B01001_038E + B01001_039E + B01001_040E + "
                         "B01001_041E",
    ("age", "above 60"): "B01001_018E + B01001_019E + B01001_020E + "
                         "B01001_021E + B01001_022E + B01001_023E + "
                         "B01001_024E + B01001_025E + B01001_042E + "
                         "B01001_043E + B01001_044E + B01001_045E + "
                         "B01001_046E + B01001_047E + B01001_048E + "
                         "B01001_049E", 
     ("race", "white"):   "B02001_002E",
     ("race", "black"):   "B02001_003E",
     ("race", "asian"):   "B02001_005E",
     ("race", "other"):   "B02001_004E + B02001_006E + B02001_007E + "
                          "B02001_008E",
    ("sex", "male"):     "B01001_002E",
    ("sex", "female"):   "B01001_026E",
}, index_cols=['NAME'])
p_acs=p_acs.set_index('NAME')
p_acs_cat.head()

cat_name                                                    age           \
cat_value                                          19 and under 20 to 35   
NAME                                                                       
Block Group 3, Census Tract 5839, Wayne County,...          318      301   
Block Group 2, Census Tract 5839, Wayne County,...          193      163   
Block Group 1, Census Tract 5839, Wayne County,...          415      261   
Block Group 3, Census Tract 5002, Wayne County,...          147      195   
Block Group 2, Census Tract 5002, Wayne County,...          180      209   

cat_name                                                              race  \
cat_value                                          35 to 60 above 60 asian   
NAME                                                                         
Block Group 3, Census Tract 5839, Wayne County,...      497      280    31   
Block Group 2, Census Tract 5839, Wayne County,...      371      120    79   
Block Group 1, Census Tract 5839, Wayne County,...      492      404     0   
Block Group 3, Census Tract 5002, Wayne County,...      252       74    44   
Block Group 2, Census Tract 5002, Wayne County,...      159       44     0   

cat_name                                                                sex  \
cat_value                                          black other white female   
NAME                                                                          
Block Group 3, Census Tract 5839, Wayne County,...   675    10   680    630   
Block Group 2, Census Tract 5839, Wayne County,...   103    18   647    497   
Block Group 1, Census Tract 5839, Wayne County,...   160    13  1399    942   
Block Group 3, Census Tract 5002, Wayne County,...   601     0    23    371   
Block Group 2, Census Tract 5002, Wayne County,...   547    24    21    251   

cat_name                                                 
cat_value                                          male  
NAME                                                     
Block Group 3, Census Tract 5839, Wayne County,...  766  
Block Group 2, Census Tract 5839, Wayne County,...  350  
Block Group 1, Census Tract 5839, Wayne County,...  630  
Block Group 3, Census Tract 5002, Wayne County,...  297  
Block Group 2, Census Tract 5002, Wayne County,...  341

## 人口合成 Population Synthesis 

对于每一个PUMA（个体数据的空间单元），首先计算其个体属性的联合分布，然后从联合分布中抽样，使抽样结果满足从普查数据中已知的边缘分布

参考：A METHODOLOGY TO MATCH DISTRIBUTIONS OF BOTH HOUSEHOLD AND PERSON ATTRIBUTES IN THE GENERATION OF SYNTHETIC POPULATIONS

文献：http://www.scag.ca.gov/Documents/PopulationSynthesizerPaper_TRB.pdf

In [22]:
import warnings
warnings.filterwarnings('ignore')
# define the data we want 
h_pums_cols = ('serialno', 'RT', 'PUMA00', 'PUMA10',  'NP',
                            'TYPE', 'VEH', 'WIF', 'NOC', 'TEN', 'RNTP', 'BDSP', 'YBL', 'HINCP')
p_pums_cols = ('serialno', 'RT', 'PUMA00', 'PUMA10', 'AGEP', 'RAC1P', 'SEX', 'SCHL', 'COW')
all_pumas=list(set(h_acs['puma']))

# do the synthesis one PUMA at a time

all_households=pd.DataFrame()
all_persons=pd.DataFrame()

for puma in all_pumas:
    print('PUMA: ', puma)
    # get the block groups in this puma
    this_puma_ind=[i for i in range(len(h_acs)) if h_acs.iloc[i]['puma']==puma]
    #download the pums data
    p_pums=c.download_population_pums(state, puma10=puma, usecols=p_pums_cols)
    h_pums=c.download_household_pums(state, puma10=puma, usecols=h_pums_cols)
    #get the joint distribution of pums data
    h_pums, jd_households = cat.joint_distribution(h_pums,
        cat.category_combinations(h_acs_cat.columns),
        {"cars": cars_cat, "children": children_cat, 
         "income": income_cat, "workers": workers_cat, "tenure": tenure_cat})
    print(jd_households)
    p_pums, jd_persons = cat.joint_distribution(
        p_pums,
        cat.category_combinations(p_acs_cat.columns),
        {"age": age_cat, "sex": sex_cat, "race": race_cat}
    )
    # simulate households and persons for each person in each block-group of this PUMA
    for bg_ind in this_puma_ind:
        zone_name=h_acs_cat.index[bg_ind]
        geoid=state+ h_acs.loc[zone_name,'county']+h_acs.loc[zone_name,'tract']+h_acs.loc[zone_name,'block group']
        print('\n', geoid, ': ', zone_name)
        best_households, best_people, people_chisq, people_p= synthesizer.synthesize(h_acs_cat.iloc[bg_ind].transpose(), p_acs_cat.iloc[bg_ind].transpose(), jd_households, jd_persons, h_pums, p_pums,
                   marginal_zero_sub=.01, jd_zero_sub=.001, hh_index_start=0)
    #     add the puma and bg id to each HH
        best_households['NAME']=zone_name
        best_households['home_geoid']=geoid
    #     add people and HH to global list
        all_households=pd.concat([all_households, best_households])
        all_persons=pd.concat([all_persons, best_people])

PUMA:  03213
                                                    cat_id  frequency
cars        children income     tenure workers                       
none        no       gt100      owned  none              0        1.0
                                       one               1        0.0
                                       two or more       2        0.0
                                rented none              3        0.0
                                       one               4        0.0
                                       two or more       5        1.0
                     gt35-lt100 owned  none              6        5.0
                                       one               7        2.0
                                       two or more       8        2.0
                                rented none              9        0.0
                                       one              10        1.0
                                       two or more      11        0.0
       


 261635159002 :  Block Group 2, Census Tract 5159, Wayne County, Michigan
Drawing 42 households

 261635159003 :  Block Group 3, Census Tract 5159, Wayne County, Michigan
Drawing 63 households

 261635509003 :  Block Group 3, Census Tract 5509, Wayne County, Michigan
Drawing 403 households

 261635509001 :  Block Group 1, Census Tract 5509, Wayne County, Michigan
Drawing 419 households

 261635509002 :  Block Group 2, Census Tract 5509, Wayne County, Michigan
Drawing 341 households

 261635516002 :  Block Group 2, Census Tract 5516, Wayne County, Michigan
Drawing 442 households

 261635516001 :  Block Group 1, Census Tract 5516, Wayne County, Michigan
Drawing 481 households

 261635516003 :  Block Group 3, Census Tract 5516, Wayne County, Michigan
Drawing 313 households

 261635502001 :  Block Group 1, Census Tract 5502, Wayne County, Michigan
Drawing 467 households

 261635502003 :  Block Group 3, Census Tract 5502, Wayne County, Michigan
Drawing 536 households

 261635502002 :  Bloc

Drawing 483 households

 261635009002 :  Block Group 2, Census Tract 5009, Wayne County, Michigan
Drawing 267 households

 261635010003 :  Block Group 3, Census Tract 5010, Wayne County, Michigan
Drawing 394 households

 261635010002 :  Block Group 2, Census Tract 5010, Wayne County, Michigan
Drawing 313 households

 261635010001 :  Block Group 1, Census Tract 5010, Wayne County, Michigan
Drawing 215 households
PUMA:  03201
                                                    cat_id  frequency
cars        children income     tenure workers                       
none        no       gt100      owned  none              0        1.0
                                       one               1        0.0
                                       two or more       2        0.0
                                rented none              3        0.0
                                       one               4        0.0
                                       two or more       5        0.0
            


 261635881002 :  Block Group 2, Census Tract 5881, Wayne County, Michigan
Drawing 506 households

 261635881001 :  Block Group 1, Census Tract 5881, Wayne County, Michigan
Drawing 572 households

 261635882002 :  Block Group 2, Census Tract 5882, Wayne County, Michigan
Drawing 614 households

 261635882001 :  Block Group 1, Census Tract 5882, Wayne County, Michigan
Drawing 970 households

 261635625002 :  Block Group 2, Census Tract 5625, Wayne County, Michigan
Drawing 617 households

 261635625001 :  Block Group 1, Census Tract 5625, Wayne County, Michigan
Drawing 342 households

 261635635002 :  Block Group 2, Census Tract 5635, Wayne County, Michigan
Drawing 801 households

 261635635001 :  Block Group 1, Census Tract 5635, Wayne County, Michigan
Drawing 723 households

 261635635003 :  Block Group 3, Census Tract 5635, Wayne County, Michigan
Drawing 338 households

 261635637001 :  Block Group 1, Census Tract 5637, Wayne County, Michigan
Drawing 383 households

 261635637002 :  Bl


 261635775001 :  Block Group 1, Census Tract 5775, Wayne County, Michigan
Drawing 363 households

 261635775004 :  Block Group 4, Census Tract 5775, Wayne County, Michigan
Drawing 439 households

 261635775003 :  Block Group 3, Census Tract 5775, Wayne County, Michigan
Drawing 243 households

 261635775002 :  Block Group 2, Census Tract 5775, Wayne County, Michigan
Drawing 223 households

 261635797003 :  Block Group 3, Census Tract 5797, Wayne County, Michigan
Drawing 412 households

 261635797001 :  Block Group 1, Census Tract 5797, Wayne County, Michigan
Drawing 440 households

 261635797002 :  Block Group 2, Census Tract 5797, Wayne County, Michigan
Drawing 418 households

 261635766003 :  Block Group 3, Census Tract 5766, Wayne County, Michigan
Drawing 603 households

 261635766001 :  Block Group 1, Census Tract 5766, Wayne County, Michigan
Drawing 178 households

 261635766004 :  Block Group 4, Census Tract 5766, Wayne County, Michigan
Drawing 313 households

 261635766002 :  Bl

...


 261635240002 :  Block Group 2, Census Tract 5240, Wayne County, Michigan
Drawing 244 households

 261635240001 :  Block Group 1, Census Tract 5240, Wayne County, Michigan
Drawing 342 households

 261635240003 :  Block Group 3, Census Tract 5240, Wayne County, Michigan
Drawing 413 households

 261635241002 :  Block Group 2, Census Tract 5241, Wayne County, Michigan
Drawing 298 households

 261635241001 :  Block Group 1, Census Tract 5241, Wayne County, Michigan
Drawing 309 households

 261635241003 :  Block Group 3, Census Tract 5241, Wayne County, Michigan
Drawing 474 households

 261635241004 :  Block Group 4, Census Tract 5241, Wayne County, Michigan
Drawing 248 households

 261635318002 :  Block Group 2, Census Tract 5318, Wayne County, Michigan
Drawing 256 households

 261635318003 :  Block Group 3, Census Tract 5318, Wayne County, Michigan
Drawing 296 households

 261635318001 :  Block Group 1, Census Tract 5318, Wayne County, Michigan
Drawing 185 households

 261635248004 :  Bl


 261635002003 :  Block Group 3, Census Tract 5002, Wayne County, Michigan
Drawing 222 households

 261635002002 :  Block Group 2, Census Tract 5002, Wayne County, Michigan
Drawing 227 households

 261635002001 :  Block Group 1, Census Tract 5002, Wayne County, Michigan
Drawing 218 households

 261635002004 :  Block Group 4, Census Tract 5002, Wayne County, Michigan
Drawing 330 households

 261635003004 :  Block Group 4, Census Tract 5003, Wayne County, Michigan
Drawing 161 households

 261635003001 :  Block Group 1, Census Tract 5003, Wayne County, Michigan
Drawing 194 households

 261635003002 :  Block Group 2, Census Tract 5003, Wayne County, Michigan
Drawing 294 households

 261635003003 :  Block Group 3, Census Tract 5003, Wayne County, Michigan
Drawing 146 households

 261635005002 :  Block Group 2, Census Tract 5005, Wayne County, Michigan
Drawing 116 households

 261635005004 :  Block Group 4, Census Tract 5005, Wayne County, Michigan
Drawing 56 households

 261635005003 :  Blo


 261635031002 :  Block Group 2, Census Tract 5031, Wayne County, Michigan
Drawing 242 households

 261635031003 :  Block Group 3, Census Tract 5031, Wayne County, Michigan
Drawing 319 households

 261635031001 :  Block Group 1, Census Tract 5031, Wayne County, Michigan
Drawing 442 households

 261635032001 :  Block Group 1, Census Tract 5032, Wayne County, Michigan
Drawing 525 households

 261635032002 :  Block Group 2, Census Tract 5032, Wayne County, Michigan
Drawing 182 households

 261635032003 :  Block Group 3, Census Tract 5032, Wayne County, Michigan
Drawing 264 households

 261635034002 :  Block Group 2, Census Tract 5034, Wayne County, Michigan
Drawing 196 households

 261635034001 :  Block Group 1, Census Tract 5034, Wayne County, Michigan
Drawing 305 households

 261635052003 :  Block Group 3, Census Tract 5052, Wayne County, Michigan
Drawing 159 households

 261635052002 :  Block Group 2, Census Tract 5052, Wayne County, Michigan
Drawing 142 households

 261635052001 :  Bl

Drawing 232 households

 261635065001 :  Block Group 1, Census Tract 5065, Wayne County, Michigan
Drawing 177 households

 261635065002 :  Block Group 2, Census Tract 5065, Wayne County, Michigan
Drawing 325 households

 261635065003 :  Block Group 3, Census Tract 5065, Wayne County, Michigan
Drawing 271 households
PUMA:  03209


KeyboardInterrupt: 